# Libbitcoin BX/System: Elliptic Curve Math 
In this example, we demonstrate the properties of scalar and elliptic curve operations over finite fields.
<br>
<img src="images/ec_math_operations.jpg" alt="drawing" style="" width="700px"/>

### Libbitcoin-System (C++) Setup

In [1]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include","/usr/local/Cellar/zeromq/4.2.5/include")
#pragma cling add_library_path("/usr/local/lib","/usr/local/Cellar/zeromq/4.2.5/lib")
#pragma cling load("bitcoin","bitcoin-protocol","zmq","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System
#include <bitcoin/bitcoin.hpp> 

<hr style="border: 0.5px dashed #000;">


# Scalar Operations


## 1) Generate a valid `secp256k1` scalar

### 1.1) With Libbitcoin-Explorer

In [2]:
!bx seed --bit_length 128 


925b2d401a0317b9d2fad24d5d41cdbc


In [3]:
!bx ec-new 394e6ed80d23574456f4641f552a9eb7


6996e113742aa9672e679a16bf70ddcfce0db2686a118f07526dd9ec1b61f058


### 1.2) With Libbitcoin-System (C++)

* `bc::data_chunk`
* `bc::pseudo_random_fill`
* `bc::encode_base16`

In [ ]:
// Create 128bits of entropy.
bc::data_chunk my_entropy(16); // 16 bytes = 128 bits
bc::pseudo_random_fill(my_entropy);

// Print out entropy.
std::cout << bc::encode_base16(my_entropy) << std::endl;


* `bc::base16_literal`
* `bc::to_chunk`
* `bc::wallet::ec_private`

In [ ]:
// Create valid secp256k1 scalar from entropy.
auto my_prev_entropy = bc::to_chunk(bc::base16_literal("394e6ed80d23574456f4641f552a9eb7"));
bc::wallet::ec_private my_key(my_prev_entropy);

// Print out scalar.
std::cout << bc::encode_base16(my_key.secret()) << std::endl;


## 2) Demonstrate Associativity

**`(a + b) * c = a * c + b * c`**
<br>

* `a = 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b`
* `b = b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2`
* `c = 6dbcfab245c6e278659dc26ec9d989c14c223f23cd17941ab45bb04c91290cdf`

### 2.1) With Libbitcoin-Explorer

**`a + b`**

In [ ]:
!bx ec-add-secrets 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2


**`Left side: (a + b) * c`**

In [ ]:
!bx ec-multiply-secrets 3cbb3d17c72dfdd1ab125a26a917b8fbcba39e296478dfadef2a7176ca2db5cc 6dbcfab245c6e278659dc26ec9d989c14c223f23cd17941ab45bb04c91290cdf
    

**`a * c`**


In [ ]:
!bx ec-multiply-secrets 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b 6dbcfab245c6e278659dc26ec9d989c14c223f23cd17941ab45bb04c91290cdf


**`b * c`**

In [ ]:
!bx ec-multiply-secrets b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2 6dbcfab245c6e278659dc26ec9d989c14c223f23cd17941ab45bb04c91290cdf


**`Right side: a * c + b * c`**

In [ ]:
!bx ec-add-secrets 94cbd69b1e8cdf4a69a1052ad783500231ee73f8c8c3f884f9f7f1cdcb8c1800 bb86d240513123db4882163833c6fec0b45cf6fb61858de324abc35b8a8c9c80


### 2.2) With Libbitcoin-System (C++)

* `bc::ec_secret`
* `bc::ec_add`
* `bc::ec_multiply`

In [ ]:
auto scalar_a = bc::base16_literal("86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b");
auto scalar_b = bc::base16_literal("b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2");
auto scalar_c = bc::base16_literal("6dbcfab245c6e278659dc26ec9d989c14c223f23cd17941ab45bb04c91290cdf");

// Left side: (a + b) * c
bc::ec_secret scalar_result(scalar_a);
bc::ec_add(scalar_result, scalar_b);    // a += b
bc::ec_multiply(scalar_result, scalar_c);  // (a + b) *= c

// Right side: a * c + b * c
bc::ec_secret scalar_a_mult_c(scalar_a);
bc::ec_secret scalar_b_mult_c(scalar_b);
bc::ec_multiply(scalar_a_mult_c, scalar_c); // a *= c
bc::ec_multiply(scalar_b_mult_c, scalar_c); // a *= c
bc::ec_secret scalar_result_(scalar_a_mult_c);
bc::ec_add(scalar_result_, scalar_b_mult_c); // (a * c) += (b * c)

// Check for equality.
std::cout << (scalar_result == scalar_result_) << std::endl;  

// Print out.
std::cout << bc::encode_base16(bc::to_chunk(scalar_result)) << std::endl;
std::cout << bc::encode_base16(bc::to_chunk(scalar_result_)) << std::endl;


<hr style="border: 0.5px dashed #000;">

# Elliptic Curve Operations:



## 3) Generate a valid `secp256k1` point

### 3.1) With Libbitcoin-Explorer

In [ ]:
!bx seed -b 128 | bx ec-new | bx ec-to-public


### 3.2) With Libbitcoin-System (C++)

* `bc::wallet::ec_private::to_public()`
* `bc::Wallet::ec_public::point()`
* `bc::ec_compressed`

In [ ]:
bc::data_chunk my_entropy_(16); // 16 bytes = 128 bits
bc::pseudo_random_fill(my_entropy_);
bc::wallet::ec_private my_key_(my_entropy_);
auto my_point = my_key_.to_public().point(); // Derive point

std::cout << bc::encode_base16(my_point) << std::endl;


## 4) Demonstrate Associativity

**`(a + b) * G = A + B`**

* `a = 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b`
* `b = b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2`

### 4.1) With Libbitcoin-Explorer


**`a + b`**


In [ ]:
!bx ec-add-secrets 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2


**`Left side: (a + b) * G`**

In [ ]:
!bx ec-to-public 3cbb3d17c72dfdd1ab125a26a917b8fbcba39e296478dfadef2a7176ca2db5cc
    

**`A`**

In [ ]:
!bx ec-to-public 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b


**`Right side: A + B`**

* `ec-add POINT secret = POINT + (secret * G)`

In [ ]:
!bx ec-add 032e053dc83eb3f4d52490ec2745d8796f1a3f5762166a0447a4e4640db5ca56d9 b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2


### 4.2) With Libbitcoin-System (C++)


In [ ]:
// Left side: (a + b) * G 
bc::ec_secret scalar_a_plus_b(scalar_a);
bc::ec_add(scalar_a_plus_b, scalar_b);
bc::ec_compressed point;
bc::secret_to_public(point, scalar_a_plus_b);

// Right side: = A + B
bc::ec_compressed point_;
bc::secret_to_public(point_, scalar_a);
bc::ec_add(point_, scalar_b);

// Check for equality.
std::cout << (point == point_) << std::endl;

// Print out points.
std::cout << bc::encode_base16(bc::to_chunk(point)) << std::endl;
std::cout << bc::encode_base16(bc::to_chunk(point_)) << std::endl;
